In [1]:
import numpy as np
from sklearn.decomposition import PCA
from scipy.linalg import sqrtm

In [10]:
def get_G(theta):
    res = np.empty([2,2])
    res[0, 0] = theta[1]**2 - theta[0]
    res[0, 1] = -theta[0] * theta[1]
    res[1, 0] = -theta[0] * theta[1]
    res[1, 1] = theta[0]**2
    return - res / (2 * theta[0]**3)

def get_G_inv(eta):
    res = np.empty([len(eta), len(eta)])
    res[0, 0] = 1/2
    res[0, 1] = - eta[1]
    res[1, 0] = - eta[1]
    res[1, 1] = eta[0] + eta[1]**2
    return res / (eta[0] - eta[1]**2)**2


def get_G_dagger(thetas):
    res = get_G(thetas[0, :])
    for theta in thetas[1:, :]:
        res += get_G(theta)
    return res

def get_lambda_(theta):
    return np.linalg.det(get_G(theta))

def get_xi(i, thetas):
    return np.sqrt(get_lambda_(thetas[i, :])) * sqrtm(get_G_dagger(thetas)) @ thetas[i, :]

def get_xis(thetas):
    res = [get_xi(0, thetas)]
    for i in range(1, n):
        res.append(get_xi(i, thetas))
    return np.array(res)




In [11]:
def get_K(h, xis):
    pca = PCA(n_components=2)
    res = pca.fit_transform(xis)
    return res[:h, :].T

def get_U_U_star(G_dagger, xis):
    pca = PCA(n_components=2)
    K_all = pca.fit_transform(xis).T
    return np.linalg.inv(sqrtm(G_dagger)) @ K_all

def get_V_V_star(U_U_star):
    return np.linalg.pinv(U_U_star)




In [12]:
n = 10
mus = np.array([np.random.rand()*2 - 1 for mu in range(n)])
sigmas = np.array([np.random.rand() + 0.5 for mu in range(n)])

# mus = np.array([0])
# sigmas = np.array([1])

theta_1s = - 1 / (2 * sigmas**2)
theta_2s = mus / sigmas**2
thetas = np.array([theta_1s, theta_2s]).T

eta_1s = mus**2 + sigmas**2
eta_2s = mus
etas = np.array([eta_1s, eta_2s]).T


In [13]:
h = 1
# Get initial U and w_tilda
u_0 = np.array([get_lambda_(theta) * theta for theta in thetas])
u_0 = sum(u_0) / sum([get_lambda_(theta) for theta in thetas])

U = np.linalg.inv(sqrtm(get_G_dagger(thetas))) @ get_K(h, get_xis(thetas))

w_tildas = [1/np.sqrt(get_lambda_(thetas[0, :])) * get_K(h, get_xis(thetas)).T @ get_xi(0, thetas)]
for i in range(1, n):
    w_tildas.append(1/np.sqrt(get_lambda_(thetas[i, :])) * get_K(h, get_xis(thetas)).T @ get_xi(i, thetas))
w_tildas = np.array(w_tildas)

In [14]:
import plotly.graph_objects as go


In [15]:
coord = np.einsum('ij,kj->ik', w_tildas, U) + u_0

In [16]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=thetas[:, 0], y=thetas[:, 1], mode='markers'))
fig.add_trace(go.Scatter(x=coord[:, 0], y=coord[:, 1], mode='markers+lines'))
fig.add_trace(go.Scatter(x=[u_0[0]], y=[u_0[1]], mode='markers'))

In [ ]:
np.sort(coord)

array([[-2.3751104 ,  0.96014049],
       [-2.034069  ,  0.79326204],
       [-1.46918729,  0.51685398],
       [-1.41092251,  0.48834384],
       [-5.95667948,  2.71267475],
       [-6.08079752,  2.77340822],
       [-1.81402276,  0.68558895],
       [-1.31656813,  0.44217433],
       [-1.35585195,  0.4613967 ],
       [-1.7619373 ,  0.66010248]])

In [ ]:
U

array([[-0.29806832],
       [ 0.14585086]])